<a href="https://colab.research.google.com/github/GUILLENJV/62f1ab79cc287b001f643389/blob/master/Grid_Search_Hyperparameters_Binary_model_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grid Search Hyperparameters Binary Model 

In [ ]:
# example of learned embedding encoding for a neural network
from numpy import unique
import numpy
from numpy import array
import pandas as pd
import datetime 
from time import sleep
from tqdm import tqdm
import os
from pandas import read_csv
from sklearn.model_selection import train_test_split
import json
import tensorflow as tf
from tensorflow import keras
import requests
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from tensorflow.keras.layers import BatchNormalization
from sklearn.preprocessing import StandardScaler
from keras.regularizers import l1
from keras.regularizers import l2
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from keras.utils.vis_utils import plot_model
from matplotlib import pyplot
import matplotlib.pyplot as plt
import seaborn as sns
# Use scikit-learn to grid search the batch size and epochs
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
# carga los datos
df = pd.read_excel('62f1ab79cc287b001f643389_cinco.xlsx') 
df = df.drop_duplicates()
# Imputamos la variable 
df['visitor_tokens_adh'].fillna(df['visitor_tokens_adh'].mode()[0], inplace=True)
df['visitor_tokens_cadid'].fillna(df['visitor_tokens_cadid'].mode()[0], inplace=True)
df['visitor_geo_location_isp'].fillna(df['visitor_geo_location_isp'].mode()[0], inplace=True)
df['visitor_geo_location_connection_type'].fillna(df['visitor_geo_location_connection_type'].mode()[0], inplace=True)
df['visitor_geo_location_cityName'].fillna(df['visitor_geo_location_cityName'].mode()[0], inplace=True)

df = df.drop(['landing_pages_group_id','visitor_tokens_add'], axis=1)

X = df[['campaign_id', 'traffic_source_id',
       'visitor_device_browser', 'visitor_device_hardware_family',
       'visitor_device_hardware_model', 'visitor_device_hardware_vendor',
       'visitor_device_os_family', 'visitor_device_os_vendor',
       'visitor_device_os_version', 'visitor_device_type',
       'visitor_geo_location_cityName', 'visitor_geo_location_connection_type',
       'visitor_geo_location_countryCode', 'visitor_geo_location_isp',
       'visitor_geo_location_regionName', 'visitor_tokens_adh',
       'visitor_tokens_cadid', 'visitor_tokens_adi', 'converted_yes',
       'converted_no']]

X = X.astype(str).to_numpy()
y = df[["landing_page_id"]].to_numpy()

# encode string input values as integers
encoded_x = None
for i in range(0, X.shape[1]):
    label_encoder = LabelEncoder()
    feature = label_encoder.fit_transform(X[:,i])
    feature = feature.reshape(X.shape[0], 1)
    onehot_encoder = OrdinalEncoder()
    feature = onehot_encoder.fit_transform(feature)
    if encoded_x is None:
        encoded_x = feature
    else:
        encoded_x = numpy.concatenate((encoded_x, feature), axis=1)
#print("X shape: : ", encoded_x.shape)

# encode string class values as integers
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(y)
label_encoded_y = label_encoder.transform(y)

# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(encoded_x, label_encoded_y, test_size=0.30, random_state=123)

scaler=StandardScaler()
scaler.fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# 1. Batch Size and Number of Epochs

In [ ]:
# Definición del modelo
def modelo_mlp(): 
    model = Sequential()
    model.add(Dense(25, input_dim=X_train.shape[1], 
                kernel_initializer= 'he_uniform',
                activation='relu', activity_regularizer=regularizers.l1(1e-4)))
    model.add(BatchNormalization())
    model.add(Dense(10, kernel_initializer= 'he_uniform',
                activation='relu',activity_regularizer=regularizers.l1(1e-4)))
    model.add(BatchNormalization())
    model.add(Dense(5, kernel_initializer= 'he_uniform',
                activation='relu',activity_regularizer=regularizers.l1(1e-4)))  
    model.add(BatchNormalization())
    model.add(Dense(1, kernel_initializer= 'he_uniform',
                activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',
                metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
# create model
model = KerasClassifier(build_fn=modelo_mlp, verbose=0)
# define the grid search parameters
batch_size = [32, 64, 128]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 25)                525       
                                                                 
 batch_normalization (BatchN  (None, 25)               100       
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 10)                260       
                                                                 
 batch_normalization_1 (Batc  (None, 10)               40        
 hNormalization)                                                 
                                                                 
 dense_2 (Dense)             (None, 5)                 55        
                                                                 
 batch_normalization_2 (Batc  (None, 5)                2

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.883088 using {'batch_size': 32, 'epochs': 100}
0.775279 (0.014279) with: {'batch_size': 32, 'epochs': 10}
0.881755 (0.012693) with: {'batch_size': 32, 'epochs': 50}
0.883088 (0.022655) with: {'batch_size': 32, 'epochs': 100}
0.763536 (0.053847) with: {'batch_size': 64, 'epochs': 10}
0.880463 (0.031102) with: {'batch_size': 64, 'epochs': 50}
0.860799 (0.049079) with: {'batch_size': 64, 'epochs': 100}
0.643952 (0.050065) with: {'batch_size': 128, 'epochs': 10}
0.864689 (0.020471) with: {'batch_size': 128, 'epochs': 50}
0.871235 (0.007888) with: {'batch_size': 128, 'epochs': 100}


#2. Training Optimization Algorithm

In [ ]:
# Definición del modelo
def modelo_mlp(optimizer='adam'): 
    model = Sequential()
    model.add(Dense(25, input_dim=X_train.shape[1], 
                kernel_initializer= 'he_uniform',
                activation='relu', activity_regularizer=regularizers.l1(1e-4)))
    model.add(BatchNormalization())
    model.add(Dense(10, kernel_initializer= 'he_uniform',
                activation='relu',activity_regularizer=regularizers.l1(1e-4)))
    model.add(BatchNormalization())
    model.add(Dense(5, kernel_initializer= 'he_uniform',
                activation='relu',activity_regularizer=regularizers.l1(1e-4)))  
    model.add(BatchNormalization())
    model.add(Dense(1, kernel_initializer= 'he_uniform',
                activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',
                metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
# create model
model = KerasClassifier(build_fn=modelo_mlp, epochs=100, batch_size=32, verbose=0)
# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 25)                525       
                                                                 
 batch_normalization (BatchN  (None, 25)               100       
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 10)                260       
                                                                 
 batch_normalization_1 (Batc  (None, 10)               40        
 hNormalization)                                                 
                                                                 
 dense_2 (Dense)             (None, 5)                 55        
                                                                 
 batch_normalization_2 (Batc  (None, 5)                2

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.904122 using {'optimizer': 'Adam'}
0.889686 (0.036401) with: {'optimizer': 'SGD'}
0.884406 (0.027209) with: {'optimizer': 'RMSprop'}
0.881776 (0.031546) with: {'optimizer': 'Adagrad'}
0.884411 (0.026035) with: {'optimizer': 'Adadelta'}
0.904122 (0.027612) with: {'optimizer': 'Adam'}
0.889670 (0.033759) with: {'optimizer': 'Adamax'}
0.901481 (0.019947) with: {'optimizer': 'Nadam'}


# 3. Learning Rate and Momentum

In [ ]:
# Definición del modelo
def modelo_mlp(learn_rate=0.01, momentum=0): 
    model = Sequential()
    model.add(Dense(25, input_dim=X_train.shape[1], 
                kernel_initializer= 'he_uniform',
                activation='relu', activity_regularizer=regularizers.l1(1e-4)))
    model.add(BatchNormalization())
    model.add(Dense(10, kernel_initializer= 'he_uniform',
                activation='relu',activity_regularizer=regularizers.l1(1e-4)))
    model.add(BatchNormalization())
    model.add(Dense(5, kernel_initializer= 'he_uniform',
                activation='relu',activity_regularizer=regularizers.l1(1e-4)))  
    model.add(BatchNormalization())
    model.add(Dense(1, kernel_initializer= 'he_uniform',
                activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',
                metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
# create model
model = KerasClassifier(build_fn=modelo_mlp, epochs=100, batch_size=32, verbose=0)
# define the grid search parameters
learn_rate = [0.0001,0.001, 0.01, 0.1]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8]
param_grid = dict(learn_rate=learn_rate, momentum=momentum)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 25)                525       
                                                                 
 batch_normalization (BatchN  (None, 25)               100       
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 10)                260       
                                                                 
 batch_normalization_1 (Batc  (None, 10)               40        
 hNormalization)                                                 
                                                                 
 dense_2 (Dense)             (None, 5)                 55        
                                                                 
 batch_normalization_2 (Batc  (None, 5)                2

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.904137 using {'learn_rate': 0.0001, 'momentum': 0.2}
0.881781 (0.028798) with: {'learn_rate': 0.0001, 'momentum': 0.0}
0.904137 (0.035177) with: {'learn_rate': 0.0001, 'momentum': 0.2}
0.890972 (0.021738) with: {'learn_rate': 0.0001, 'momentum': 0.4}
0.889650 (0.016851) with: {'learn_rate': 0.0001, 'momentum': 0.6}
0.898857 (0.021272) with: {'learn_rate': 0.0001, 'momentum': 0.8}
0.889676 (0.035252) with: {'learn_rate': 0.001, 'momentum': 0.0}
0.897560 (0.030555) with: {'learn_rate': 0.001, 'momentum': 0.2}
0.883083 (0.020766) with: {'learn_rate': 0.001, 'momentum': 0.4}
0.897560 (0.032521) with: {'learn_rate': 0.001, 'momentum': 0.6}
0.890983 (0.027959) with: {'learn_rate': 0.001, 'momentum': 0.8}
0.885723 (0.030545) with: {'learn_rate': 0.01, 'momentum': 0.0}
0.879156 (0.027938) with: {'learn_rate': 0.01, 'momentum': 0.2}
0.900174 (0.025671) with: {'learn_rate': 0.01, 'momentum': 0.4}
0.890993 (0.032719) with: {'learn_rate': 0.01, 'momentum': 0.6}
0.877849 (0.030518) with: {'

#4. Network Weight Initialization

In [ ]:
# Definición del modelo
def modelo_mlp(init_mode='uniform'): 
    model = Sequential()
    model.add(Dense(25, input_dim=X_train.shape[1], 
                kernel_initializer= 'he_uniform',
                activation='relu', activity_regularizer=regularizers.l1(1e-4)))
    model.add(BatchNormalization())
    model.add(Dense(10, kernel_initializer= 'he_uniform',
                activation='relu',activity_regularizer=regularizers.l1(1e-4)))
    model.add(BatchNormalization())
    model.add(Dense(5, kernel_initializer= 'he_uniform',
                activation='relu',activity_regularizer=regularizers.l1(1e-4)))  
    model.add(BatchNormalization())
    model.add(Dense(1, kernel_initializer= 'he_uniform',
                activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',
                metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
# create model
model = KerasClassifier(build_fn=modelo_mlp, epochs=100, batch_size=32, verbose=0)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 25)                525       
                                                                 
 batch_normalization (BatchN  (None, 25)               100       
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 10)                260       
                                                                 
 batch_normalization_1 (Batc  (None, 10)               40        
 hNormalization)                                                 
                                                                 
 dense_2 (Dense)             (None, 5)                 55        
                                                                 
 batch_normalization_2 (Batc  (None, 5)                2

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.906736 using {'init_mode': 'uniform'}
0.906736 (0.022476) with: {'init_mode': 'uniform'}
0.884374 (0.010210) with: {'init_mode': 'lecun_uniform'}
0.900195 (0.037170) with: {'init_mode': 'normal'}
0.884354 (0.006851) with: {'init_mode': 'zero'}
0.879115 (0.004710) with: {'init_mode': 'glorot_normal'}
0.890978 (0.024834) with: {'init_mode': 'glorot_uniform'}
0.902809 (0.027980) with: {'init_mode': 'he_normal'}
0.896217 (0.021777) with: {'init_mode': 'he_uniform'}


#5. Neuron Activation Function

In [ ]:
# Definición del modelo
def modelo_mlp(activation='relu'): 
    model = Sequential()
    model.add(Dense(25, input_dim=X_train.shape[1], 
                kernel_initializer= 'he_uniform',
                activation='relu', activity_regularizer=regularizers.l1(1e-4)))
    model.add(BatchNormalization())
    model.add(Dense(10, kernel_initializer= 'he_uniform',
                activation='relu',activity_regularizer=regularizers.l1(1e-4)))
    model.add(BatchNormalization())
    model.add(Dense(5, kernel_initializer= 'he_uniform',
                activation='relu',activity_regularizer=regularizers.l1(1e-4)))  
    model.add(BatchNormalization())
    model.add(Dense(1, kernel_initializer= 'he_uniform',
                activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',
                metrics=['accuracy'])
    #model.summary()
    return model

In [ ]:
# create model
model = KerasClassifier(build_fn=modelo_mlp, epochs=100, batch_size=32, verbose=0)
# define the grid search parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 25)                525       
                                                                 
 batch_normalization_3 (Batc  (None, 25)               100       
 hNormalization)                                                 
                                                                 
 dense_5 (Dense)             (None, 10)                260       
                                                                 
 batch_normalization_4 (Batc  (None, 10)               40        
 hNormalization)                                                 
                                                                 
 dense_6 (Dense)             (None, 5)                 55        
                                                                 
 batch_normalization_5 (Batc  (None, 5)               

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.909361 using {'activation': 'softmax'}
0.909361 (0.017782) with: {'activation': 'softmax'}
0.897534 (0.033333) with: {'activation': 'softplus'}
0.889655 (0.019400) with: {'activation': 'softsign'}
0.875230 (0.037403) with: {'activation': 'relu'}
0.887020 (0.016335) with: {'activation': 'tanh'}
0.889676 (0.030877) with: {'activation': 'sigmoid'}
0.890978 (0.027217) with: {'activation': 'hard_sigmoid'}
0.900148 (0.009118) with: {'activation': 'linear'}


#6. Number of Neurons in the Hidden Layer

In [ ]:
# Definición del modelo
def modelo_mlp(neurons=1): 
    model = Sequential()
    model.add(Dense(25, input_dim=X_train.shape[1], 
                kernel_initializer= 'he_uniform',
                activation='relu', activity_regularizer=regularizers.l1(1e-4)))
    model.add(BatchNormalization())
    model.add(Dense(10, kernel_initializer= 'he_uniform',
                activation='relu',activity_regularizer=regularizers.l1(1e-4)))
    model.add(BatchNormalization())
    model.add(Dense(5, kernel_initializer= 'he_uniform',
                activation='relu',activity_regularizer=regularizers.l1(1e-4)))  
    model.add(BatchNormalization())
    model.add(Dense(1, kernel_initializer= 'he_uniform',
                activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam',
                metrics=['accuracy'])
    #model.summary()
    return model

In [ ]:
# create model
model = KerasClassifier(build_fn=modelo_mlp, epochs=100, batch_size=32, verbose=0)
# define the grid search parameters
neurons = [1, 3, 5, 10]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  


In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.908064 using {'neurons': 3}
0.896227 (0.025874) with: {'neurons': 1}
0.908064 (0.027619) with: {'neurons': 3}
0.906746 (0.028899) with: {'neurons': 5}
0.892290 (0.029057) with: {'neurons': 10}
